# ETL

## IMPORTANDO LIBRERIAS 

In [410]:

import pandas as pd
import numpy as np
# Módulo para trabajar con rutas de archivo
from pathlib import Path
# Librería para identificar encondings
import chardet
#importamos lalibreria os para trabajar con los archivos
import os


## cargando data set

In [411]:
#funcion para leer archivos csv y json a partir de una ruta
def get_data(filepath):
    #manejando los archivos csv y json
    with open(filepath, 'rb') as file:
        result = chardet.detect(file.read())
        # Para importar archivos csv, detectando encoding
        if Path(filepath).suffix == ".csv": 
            DF = pd.read_csv(filepath, encoding=result['encoding'])
        # Para importar archivos json
        elif Path(filepath).suffix == ".json": 
            DF = pd.read_json(filepath) 

    return DF

#en esta funcion podemos añadir los archivos que queremos leer segun nuestras necesidades

In [412]:
#funcionque nos da el nombres de los archivos a partir de la ruta del   archivo
def get_nombre_archivo(filepath):
    #leemos el contenido de la ruta 
    archivos = os.listdir(filepath)
    archivos_sin_extension = [os.path.splitext(nombre_archivo)[0] for nombre_archivo in archivos if os.path.splitext(nombre_archivo)[1] in ['.csv','.json'] ]
    return archivos_sin_extension

In [414]:
#creando  los data framees a partir de los nombres que nos dieron
i=1
directorio='./data/'

for nombre_archivo in get_nombre_archivo(directorio):
    locals()[nombre_archivo]=get_data(f'{directorio}{nombre_archivo}.csv')
    print(f'el nombre de tu variable {i} es :',nombre_archivo)
    i+=1

el nombre de tu variable 1 es : disney_plus_titles
el nombre de tu variable 2 es : amazon_prime_titles
el nombre de tu variable 3 es : netflix_titles
el nombre de tu variable 4 es : hulu_titles


In [415]:

#bucle para ver el esquema de las df 
for df in get_nombre_archivo(directorio):
    print('-'*50)
    print(f'las caractericas del df {df} es :',)
    print('-'*50)
    print(locals()[df].info())

--------------------------------------------------
las caractericas del df disney_plus_titles es :
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB
None
--------------------------------------------------
las caractericas del df amazon_prime_titles es

In [416]:
amazon_prime_titles.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7585 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


### Inputacion de datos

In [417]:
for df in get_nombre_archivo(directorio):
    locals()[df].set_index('title', drop=True, inplace=True)

In [418]:
for df in get_nombre_archivo(directorio):
    for data in get_nombre_archivo(directorio):
        if data != df:
            locals()[df].fillna(locals()[data], inplace=True)

for df in get_nombre_archivo(directorio):
    locals()[df].reset_index(inplace=True)

## Transformaciones

In [419]:
# Generando el campo id
for df in get_nombre_archivo(directorio):
    locals()[df]['id'] = df[0]+locals()[df]['show_id']

In [420]:
#Remplazando los valores nulos por G
for df in get_nombre_archivo(directorio):
    locals()[df]['rating'].fillna('G', inplace=True)

In [421]:

#dando  formato AAAA-mm-dd a las fechas 
for df in get_nombre_archivo(directorio):
    #formato de las fechasoriginales 
    date_format ='%B %d, %Y'
    #convertimos las fechas a formato AAAA-mm-dd
    locals()[df]['date_added']=pd.to_datetime(locals()[df]['date_added'].str.strip(),format=date_format)
    #con esta linea de codigo podemos darle el formato que queramos a las fechas
    #locals()[df]['date_added'] = locals()[df]['date_added'].strftime("%d %B %Y")



In [422]:
#Los campos de texto deberán estar en minúsculas, sin excepciones
for df in get_nombre_archivo(directorio):
    colunas_obj=locals()[df].select_dtypes('object').columns
    locals()[df][colunas_obj]=(
                                locals()[df][colunas_obj]
                               .apply(lambda x: x.str.lower())
                               )

In [426]:
#El campo duration debe convertirse en dos campos: duration_int y duration_unit
hulu_titles.fillna('sin dato', inplace=True)
hulu_titles['duration_int']=hulu_titles['duration'].apply(lambda x: x.split(' ')[0])
hulu_titles['duration_int'].replace('sin', '0', inplace=True)
hulu_titles['duration_int']=hulu_titles['duration_int'].astype(int)

In [427]:
hulu_titles['duration_type']=hulu_titles['duration'].apply(lambda x: x.split(' ')[1])

In [428]:
hulu_titles

,show_id,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,duration_int,duration_type
title,,,,,,,,,,,,,,
Ricky Velez: Here's Everything,s1,movie,sin dato,sin dato,sin dato,2021-10-24 00:00:00,2021,tv-ma,sin dato,"comedy, stand up",​comedian ricky velez bares it all with his ho...,hs1,0,dato
Silent Night,s2,movie,sin dato,sin dato,sin dato,2021-10-23 00:00:00,2020,g,94 min,"crime, drama, thriller","mark, a low end south london hitman recently r...",hs2,94,min
The Marksman,s3,movie,marcus adams,"wesley snipes, william hope, emma samms, antho...",sin dato,2021-10-23 00:00:00,2021,pg-13,108 min,"action, thriller",a hardened arizona rancher tries to protect an...,hs3,108,min
Gaia,s4,movie,sin dato,sin dato,sin dato,2021-10-22 00:00:00,2021,r,97 min,horror,a forest ranger and two survivalists with a cu...,hs4,97,min
Settlers,s5,movie,sin dato,sin dato,sin dato,2021-10-22 00:00:00,2021,g,104 min,"science fiction, thriller",mankind's earliest settlers on the martian fro...,hs5,104,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Star Trek: The Original Series,s3069,tv show,sin dato,sin dato,united states,sin dato,1966,tv-pg,3 seasons,"action, adventure, classics",the 23rd century adventures of captain james t...,hs3069,3,seasons
Star Trek: Voyager,s3070,tv show,sin dato,"kate mulgrew, robert beltran, roxann biggs-daw...",united states,2017-10-01 00:00:00,1997,tv-pg,7 seasons,"action, adventure, science fiction",catapulted into the distant sector of the gala...,hs3070,7,seasons
The Fades,s3071,tv show,sin dato,"iain de caestecker, daniel kaluuya, tom ellis,...",united kingdom,sin dato,2011,tv-14,1 season,"horror, international, science fiction",seventeen-year-old paul is haunted by apocalyp...,hs3071,1,season


# Exportando los datos 